In [1]:
!nvidia-smi

Wed Apr 12 06:57:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0D:00.0 Off |                  N/A |
| 23%   26C    P2    49W / 250W |      0MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [4]:
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.networkx.blocks.ham import HAM
from networks.ssl_head import SSLHead

# sync python module
%load_ext autoreload
%autoreload 2

In [6]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.4
args.depths = [3, 3, 9, 3]
net = None
del net
net = network('unetcnx_a0_s1', args) # unetc2f
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unetcnx_a0_s1
patch size: 4
drop rate: 0.4
depths: [3, 3, 9, 3]


TypeError: __init__() got an unexpected keyword argument 'reduction'

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))